In [1]:
import pandas as pd
from py2neo import Graph
from py2neo.data import Node, Relationship

### Recupero dati da db

In [2]:
#Cloud
graph = Graph(scheme="bolt", host="54.90.165.10", port="7687", user="neo4j", password="clicks-haul-qualifier")

In [3]:
#localhost
#graph = Graph(scheme="bolt", host="localhost", port="7787", user="neo4j", password="neo4j1")

<br/>
Autori di submissions e comments contenenti specifiche parole chiave

In [4]:
query = '''
WITH [ "🚀","gme", "gamestop", "gamestonk", "squeeze", "robinhood", "everyone", "moon"] AS terms
MATCH (t:Author)-[x:COMMENTED]->(c:Comment)
WITH t, terms, c
MATCH (t:Author)-[y:SUBMITTED]->(s:Submission)
WHERE any(word IN terms WHERE toLower(c.title) CONTAINS word)
OR any(word IN terms WHERE toLower(s.title) CONTAINS word)
RETURN DISTINCT t.name, t.degree
'''
result = graph.run(query)
authors = pd.DataFrame(result)
authors.rename(columns={0:'name', 1:'degree'}, inplace=True)

In [5]:
len(authors.index)

927

In [6]:
authors.sort_values(by='degree', ascending=False)

,name,degree
49,[deleted],392.0
108,btoned,40.0
190,PencesElectrician,32.0
121,iTradeStalks,32.0
593,FudgieThaWhale,31.0
...,...,...
596,JeremyLinForever,2.0
165,ThatBostonGuy87,2.0
594,flying_ina_metaltube,2.0
592,Trowawaycausebanned4,2.0


In [7]:
authors.to_csv('../data/from_db/reddit_authors.csv', sep=',', index=False)

<br/>
Submissions relative a gamestop: recupero delle submissions con il testo contenente alcune parole chiave riferite all'operazione di corsa a  rialzo del titolo di GameStop GME

In [8]:
query = '''
WITH [ "🚀","gme", "gamestop", "gamestonk", "squeeze", "robinhood", "everyone", "moon"] AS terms
match (t:Submission)
WHERE any(word IN terms WHERE toLower(t.title) CONTAINS word)
RETURN
t.total_awards_received,
t.author,
t.flair,
t.num_comments,
t.score,
t.title,
t.upvote_ratio,
t.no_follow,
t.id,
t.created_at_utc,
t.degree,
t.author_premium,
t.subreddit_subscribers
'''
result = graph.run(query)
submissions = pd.DataFrame(result)
submissions.rename(columns={0:'total_awards_received', 1:'author', 2:'flair', 3:'num_comments',
                           4:'score', 5:'title', 6:'upvote_ratio', 7:'no_follow',
                           8:'id', 9:'created_at_utc', 10:'degree', 11:'author_premium',
                           12:'subreddit_subscribers'}
                   , inplace=True)

In [9]:
submissions.to_csv('../data/from_db/reddit_submissions.csv', sep=',', index=False)

In [10]:
submissions

,total_awards_received,author,flair,num_comments,score,title,upvote_ratio,no_follow,id,created_at_utc,degree,author_premium,subreddit_subscribers
0,2,Jeffamazon,Meme,377,35,$GME - End Game 🚀🚀🚀🚀🚀,1.00,False,kwuc3a,2021-01-14 00:42:47+00:00,0.0,True,1822935
1,0,caverypca,Gain,60,25,Thanks GME weeklies!,1.00,False,kwtphr,2021-01-14 00:08:53+00:00,0.0,False,1822438
2,0,TexasOilYachts,Discussion,30,23,The First Annual GameStop Appreciation Day!,0.92,False,kwx9lu,2021-01-14 03:27:41+00:00,0.0,True,1824982
3,0,EasyReddit23,Gain,34,23,GME just nearly doubled this Brit's life savings,1.00,False,kwtz8t,2021-01-14 00:23:15+00:00,0.0,False,1822653
4,0,Gausauce,Discussion,12,22,Thank you GME gang and thank you to the rest o...,1.00,True,kwuswf,2021-01-14 01:08:20+00:00,0.0,False,1823295
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5166,0,RiverOfNexus,Discussion,12,1,"I Spoke With A Gamestop Employee, GME To The M...",1.00,True,l0683d,2021-01-18 23:04:35+00:00,0.0,False,1888957
5167,0,ExaltedDLo,Gain,2,1,"$BB + 6% on TSX. Listen up, my southern autist...",1.00,True,l04746,2021-01-18 21:24:21+00:00,0.0,False,1888074
5168,0,SFNS,Meme,7,1,TENDIE 🍗 VICTORIA - GME ANTHEM,1.00,True,l077r6,2021-01-18 23:56:19+00:00,0.0,False,1889357
5169,0,AbjectRaise,Discussion,4,1,PSA: Turn Off Stock Lending On Your Brokerage ...,1.00,True,l077vo,2021-01-18 23:56:30+00:00,0.0,False,1889358


<br/>
Comments relativi a gamestop: recupero dei comments con il testo contenente alcune parole chiave riferite all'operazione di corsa a  rialzo del titolo di GameStop GME

In [11]:
query = '''
WITH [ "🚀","gme", "gamestop", "gamestonk", "squeeze", "robinhood", "everyone", "moon"] AS terms
match (t:Comment)
WHERE any(word IN terms WHERE toLower(t.title) CONTAINS word)
RETURN
t.total_awards_received,
t.author,
t.num_comments,
t.score,
t.title,
t.upvote_ratio,
t.no_follow,
t.id,
t.link_id,
t.parent_id,
t.created_at_utc,
t.degree,
t.author_premium,
t.subreddit_subscribers
'''
result = graph.run(query)
comments = pd.DataFrame(result)
comments.rename(columns={0:'total_awards_received', 1:'author', 2:'num_comments', 3:'score',
                           4:'title', 5:'upvote_ratio', 6:'no_follow',
                           7:'id', 8:'link_id', 9:'parent_id', 10:'created_at_utc',
                           11:'degree', 12:'author_premium', 13:'subreddit_subscribers'}
                , inplace=True)

In [12]:
comments

,total_awards_received,author,num_comments,score,title,upvote_ratio,no_follow,id,link_id,parent_id,created_at_utc,degree,author_premium,subreddit_subscribers
0,0,VacationLover1,0,983,"Get off me, I’m a Robinhood hundredaire and up...",0,False,gim73ad,t3_kti7at,t3_kti7at,2021-01-09 02:55:06+00:00,0.0,True,
1,0,AK45HSR,0,128,I sold after the s&amp;p includion and then go...,0,False,giluael,t3_ktg53h,t3_ktg53h,2021-01-09 01:05:06+00:00,1.0,False,
2,0,Creepy_Web7926,0,101,I see rockets and a funny speech on my meme st...,0,False,gim0j82,t3_kth9gp,t3_kth9gp,2021-01-09 01:58:37+00:00,1.0,True,
3,1,Koosh_ed,0,91,If you don’t look at your portfolio and become...,0,False,gim2z4r,t3_kth9gp,t3_kth9gp,2021-01-09 02:19:48+00:00,1.0,False,
4,0,LandonBurrito,0,73,Change that to $STLD and you’ll be among the s...,0,False,gimrfgq,t3_kti7at,t1_gimpx9f,2021-01-09 06:17:17+00:00,1.0,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,0,spike142,0,207,The AH trading for GME already looks pretty pr...,0,False,gjamems,t3_kxhitu,t1_gja9kiq,2021-01-15 01:13:44+00:00,1.0,False,
2764,0,desertrock62,0,193,"""Openly plotting"" to defeat the shorts. What t...",0,False,gjaw4sx,t3_kxjrp1,t3_kxjrp1,2021-01-15 02:39:30+00:00,1.0,True,
2765,1,JENINTECH,0,192,This got me wet 💦💦💦🚀🚀🚀🚀,0,False,gjahdwc,t3_kxiti0,t3_kxiti0,2021-01-15 00:30:26+00:00,1.0,False,
2766,1,00001143,0,186,This is fucking beautiful. Thank you and thank...,0,False,gjb294h,t3_kxgpa3,t1_gja7e3y,2021-01-15 03:34:53+00:00,0.0,False,


In [13]:
comments.to_csv('../data/from_db/reddit_comments.csv', sep=',', index=False)

Query per relazioni tra utenti e visualizzazione grafo su Neo4J

In [ ]:
query = '''
WITH [ "🚀","gme", "gamestop", "gamestonk", "squeeze", "robinhood", "everyone", "moon"] AS terms
MATCH (s:Submission)
WHERE any(word IN terms WHERE toLower(s.title) CONTAINS word)
CALL apoc.path.subgraphNodes(s, {labelFilter:'+Author|Comment|Submission'}) 
YIELD node
RETURN node limit 400
'''
result = graph.run(query)